In [1]:
import jupyter_Importer
#from RoomOccupancy_MLModels_FeatureImportance import df_2feature_be_generates
#from MLModels_FeatureImportance import df_2feature_be_generate

import pandas as pd
import numpy as np
import random
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier    
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("Occupancy.csv", names = ['x' + str(i) for i in range(1, 7)] + ['y'], sep = ",")
dataset.head()

,x1,x2,x3,x4,x5,x6,y
0,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-02 14:19:00,23.7,26.272,585.2,749.2,0.0047641630241641,1
2,2015-02-02 14:19:59,23.718,26.29,578.4,760.4,0.0047726609921251,1
3,2015-02-02 14:21:00,23.73,26.23,572.666666666667,769.666666666667,0.0047651525524654,1
4,2015-02-02 14:22:00,23.7225,26.125,493.75,774.75,0.0047437733559968,1


In [3]:
dataset = dataset.iloc[1:,1:]
feature = ['Temperature','Humidity','Light','CO2','HumidityRatio','y']
dataset.columns = ['Temperature','Humidity','Light','CO2','HumidityRatio','y']

for i in feature :
    dataset[i] = dataset[i].apply(lambda x : float(x))
feature = ['Temperature','Humidity','Light','CO2','HumidityRatio']
dataset.head()

,Temperature,Humidity,Light,CO2,HumidityRatio,y
1,23.7000,26.272,585.200000,749.200000,0.004764,1.0
2,23.7180,26.290,578.400000,760.400000,0.004773,1.0
3,23.7300,26.230,572.666667,769.666667,0.004765,1.0
4,23.7225,26.125,493.750000,774.750000,0.004744,1.0
5,23.7540,26.200,488.600000,779.000000,0.004767,1.0


In [4]:
from sklearn import preprocessing 
dataset_origin = dataset.copy()
x = dataset_origin.iloc[:,:-1].values
x = preprocessing.scale(x)#標準化
y = dataset_origin.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=1)
clf_origin = SVC(kernel='linear',C=1,gamma='auto')
clf_origin.fit(X_train, y_train)
y_predict = clf_origin.predict(X_test)
result_train = clf_origin.score(X_train, y_train)
result_test = clf_origin.score(X_test, y_test)
print('Occupancy dataset original trained data accuracy is : '+str(result_train))
print('Occupancy dataset original test data accuracy is : '+str(result_test))




Occupancy dataset original trained data accuracy is : 0.9891780155642024
Occupancy dataset original test data accuracy is : 0.9878404669260701


In [5]:
BE = 0.2
ConfidenceScore_test = []
ConfidenceScore_train = []
#dataset_BE = dataset.copy()
#x = dataset_BE.iloc[:,:-1].values
#x = preprocessing.scale(x)#標準化
#y = dataset_BE.iloc[:,-1].values

for i in feature :
    dataset_BE = dataset.copy()
    err = dataset_BE[i].mean()
    for idx,cell in dataset_BE[i].iteritems():
        rd = random.random()
        if rd > 0.5 :
            dataset_BE[i][idx] += err
        else :
            dataset_BE[i][idx] -= err
    
    x = dataset_BE.iloc[:,:-1].values
    x = preprocessing.scale(x)
    y = dataset_BE.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=1)
    clf_BE = SVC(kernel='linear',C=1,gamma='auto')
    clf_BE.fit(X_train, y_train)
    y_predict = clf_BE.predict(X_test)
    result_train = clf_BE.score(X_train, y_train)
    result_test = clf_BE.score(X_test, y_test)
    print('Occupancy dataset with BE in feature['+str(i)+'] trained data accuracy is : '+str(result_train))
    print('Occupancy dataset with BE in feature['+str(i)+'] test data accuracy is : '+str(result_test))
    ConfidenceScore_test.append(result_test)
    ConfidenceScore_train.append(result_train)

print(ConfidenceScore_train)
print(ConfidenceScore_test)





Occupancy dataset with BE in feature[Temperature] trained data accuracy is : 0.9891780155642024
Occupancy dataset with BE in feature[Temperature] test data accuracy is : 0.9875972762645915
Occupancy dataset with BE in feature[Humidity] trained data accuracy is : 0.9890564202334631
Occupancy dataset with BE in feature[Humidity] test data accuracy is : 0.9880836575875487
Occupancy dataset with BE in feature[Light] trained data accuracy is : 0.9719722762645915
Occupancy dataset with BE in feature[Light] test data accuracy is : 0.9730058365758755
Occupancy dataset with BE in feature[CO2] trained data accuracy is : 0.9889956225680934
Occupancy dataset with BE in feature[CO2] test data accuracy is : 0.9878404669260701
Occupancy dataset with BE in feature[HumidityRatio] trained data accuracy is : 0.9891172178988327
Occupancy dataset with BE in feature[HumidityRatio] test data accuracy is : 0.9883268482490273
[0.9891780155642024, 0.9890564202334631, 0.9719722762645915, 0.9889956225680934, 0.98

In [6]:
for i in range(5) :
    ConfidenceScore_test[i] = 0.9878404669260701 / ConfidenceScore_test[i]
print(ConfidenceScore_test) 

[1.0002462447672986, 0.9997538764459759, 1.015246188452887, 1.0, 0.999507874015748]


In [7]:
dataset_light_co2 = dataset_origin.copy()
dataset_light_co2 = dataset_light_co2.iloc[:,2:7]
dataset_light_co2 = dataset_light_co2.drop(columns=['HumidityRatio'])
#dataset_light_co2['Light'] = dataset_light_co2['Light'].apply(lambda x : float(x))
#dataset_light_co2['CO2'] = dataset_light_co2['CO2'].apply(lambda x : float(x))
#dataset_light_co2['y'] = dataset_light_co2['y'].apply(lambda x : int(x))
subset = dataset_light_co2.sample(frac=0.2,replace=False,random_state=43)
dataset_light_co2 = dataset_light_co2.append(subset)
dataset_light_co2 = dataset_light_co2.drop_duplicates(keep=False)

subsample_x = subset.iloc[:,:-1].values
subsample_y = subset.iloc[:,-1].values

error_a = []
error_b = []
light_CScore = 1.0117280381068128
CO2_CSore = 0.9885700987654558
ConfidenceSore_lightCO2 = [light_CScore/(light_CScore+CO2_CSore),CO2_CSore/(light_CScore+CO2_CSore)]
for i in range(21) :
    error_a.append(i)
    error_b.append(20-i)

#i for brute force or not,n for negative importance
def df_2feature_be_generates(BE,i,n,importance=0) :
    temp = dataset_light_co2.copy()
    df_2feature_be = pd.DataFrame.to_numpy(dataset_light_co2)
    
    if i >= 0 and n == 0 :
        e1 = temp['Light'].mean()*BE*(error_a[i]/(error_a[i]+error_b[i]))
        e2 = temp['CO2'].mean()*BE*(error_b[i]/(error_a[i]+error_b[i]))
    elif i == -1 and n == 1:
        e1 = temp['Light'].mean()*BE*importance[0]
        e2 = temp['CO2'].mean()*BE*importance[1]
    elif i == -1 and n == 0 :
        e1 = temp['Light'].mean()*BE*importance[1]
        e2 = temp['CO2'].mean()*BE*importance[0]
    
    for idx,cell in temp['Light'].iteritems():
        temp['Light'][idx] += e1
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    for idx,cell in temp['Light'].iteritems():
        temp['Light'][idx] = temp['Light'][idx] - e1
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    for idx,cell in temp['CO2'].iteritems():
        temp['CO2'][idx] += e2
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    for idx,cell in temp['CO2'].iteritems():
        temp['CO2'][idx] = temp['CO2'][idx] - e2
    temp = temp.to_numpy()
    df_2feature_be = np.vstack((df_2feature_be,temp))
    #print(df_2feature_be.shape)
    temp = dataset_light_co2.copy()

    df_2feature_be = pd.DataFrame(df_2feature_be)
    #print(df_2feature_be)
    return df_2feature_be

In [8]:
#using confidence score as error allocation rate to predict by svm
BE = 0.2
df_2feature_be = df_2feature_be_generates(BE,-1,0,ConfidenceSore_lightCO2)
x_be = df_2feature_be.iloc[:, :-1].values
x_be = preprocessing.scale(x_be)#標準化
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE = SVC(kernel='linear',C=1,gamma='auto')
clf_BE.fit(X_train, y_train)
y_BE_predict = clf_BE.predict(X_test)
result_BE_train = clf_BE.score(X_train, y_train)
result_BE_test = clf_BE.score(X_test, y_test)
subsample_x = preprocessing.scale(subsample_x)
subset_raw = clf_BE.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) using confidence score on x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = using confidence score on  x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


BESVM Accuracy (on training) using confidence score on x1 bounded error (0.2%*0.4942113780654799),x2 with error (0.2%*0.5057886219345201) : 0.971696018943318
BESVM Accuracy (on testing) = using confidence score on  x1 bounded error (0.2%*0.4942113780654799),x2 with error (0.2%*0.5057886219345201) : 0.972472990972325
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9535505836575876


In [9]:
#using confidence score as error allocation rate to predict by  knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,ConfidenceSore_lightCO2)
x_be = df_2feature_be.iloc[:, :-1].values
x_be = preprocessing.scale(x_be)#標準化
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subsample_x = preprocessing.scale(subsample_x)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKN Accuracy (on training) using confidence score on x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = using confidence score on  x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


BEKN Accuracy (on training) using confidence score on x1 bounded error (0.2%*0.4942113780654799),x2 with error (0.2%*0.5057886219345201) : 0.976209856445168
BEKNN Accuracy (on testing) = using confidence score on  x1 bounded error (0.2%*0.4942113780654799),x2 with error (0.2%*0.5057886219345201) : 0.9723249963001332
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9253404669260701


In [11]:
#importanceReg,KMeans
df_2feature_be = df_2feature_be_generates(BE,-1,0,ConfidenceSore_lightCO2)
x_be = df_2feature_be.iloc[:, :-1].values
#x_be = preprocessing.scale(x_be)#標準化
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KM_BE=KMeans(n_clusters=2,init='random',random_state=5)
KM_BE.fit(X_train)
y_BE_predict = KM_BE.predict(X_train)
y_BE_predict_test = KM_BE.predict(X_test)
#subsample_x = preprocessing.scale(subsample_x)
y_subset = KM_BE.predict(subsample_x)
result_BE_train = 1 - np.sum(y_BE_predict!=y_train)/len(y_train)
result_BE_test = 1 - np.sum(y_BE_predict_test!=y_train)/len(y_test)
subset_raw = 1 - np.sum(subsample_y!=y_subset)/len(subsample_y)
    
    
print('BEKM Accuracy (on training) using confidence score on x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_train))
print('BEKM Accuracy (on testing) = using confidence score on  x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_test))
print('BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


BEKM Accuracy (on training) using confidence score on x1 bounded error (0.2%*0.4942113780654799),x2 with error (0.2%*0.5057886219345201) : 0.726949829806127
BEKM Accuracy (on testing) = using confidence score on  x1 bounded error (0.2%*0.4942113780654799),x2 with error (0.2%*0.5057886219345201) : 0.9998520053278082
BEKM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.7665369649805447


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  
